# Exercise
In this exercise, you will create a simple ETL pipeline to extract transaction data from HTML files. The end goal is to have an SQLite DB populated from these files, where each row is an HTML file. Each HTML file is a transaction that is either successful or has been declined, along with customer information, billing information, credit card details, fees, and linking transaction info.
This notebook will act as the orchestrator, where you will import the functions and classes that you created to extract, transform, and load the data. You will use them to run the steps in this notebook. Please do not change the arguments and outputs for the functions.
Please spend no more than 4 hours on this.
# Evaluation
* use of pep8 standards for codes
* column names in DB should be in snake_case or camelCase with no special characters (check the library inflection)
* use of logging and code comments to describe logic and checks
* unit tests for important functions
* ability to finish taks
You are allowed to use any python library to complete this exercise.


## below is an example of the data needed to be extracted from the HTML
import pandas as ps

In [12]:
from IPython.core.display import HTML
with open('../transactions/0049250e7e5c4bb9b9f2bdcd23188109.html') as f:
    template_html ='\n'.join(f)
HTML(template_html)

# Import code

In [348]:
# Put your imputs here
import sys, os, sqlite3, pandas as pd
# add your package to python path
sys.path.append('../')
import src
from pathlib import Path, PurePath
from sqlite3 import connect
import logging
from bs4 import BeautifulSoup

In [349]:
# so logs will be printed on notebook
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

In [350]:
# global varibles
html_folder_path = Path('../transactions') # folder to raw html files
csv_save_path = Path('../data/transactions.csv') # folder where proceed html data will go
sqlite_file_path = Path('../data/transactions.sqlite3') # path to sql db file

# Functions to modify

In [416]:
# Extract html and turn into csv
def scrape_html_to_csv(html_folder_path:PurePath)->int:
    processed_rows = 0
    html_path = ''
    # empty list 
    data = [] 
    all_data = []
    list_header = []
    for file in os.listdir(html_folder_path):
        if file.endswith('.html'):
            processed_rows += 1
            html_path = '../transactions/' + file
            list_header, data = html_header_data(html_path)
            all_data.append(data)
    # Storing the data into Pandas 
    # DataFrame 
    dataFrame = pd.DataFrame(all_data, columns = list_header)

    # Converting Pandas DataFrame 
    # into CSV file 
    dataFrame.to_csv(csv_save_path) 

    # return the number of processed rows
    return processed_rows

In [417]:
def initalize_tables(sqlite_file_path:PurePath)->None:
    # should run condtional on the tables with the html data are not created
    list_tables= '''SELECT name FROM sqlite_temp_master WHERE type='table' '''
    con = connect(sqlite_file_path)
    cor = con.cursor()
    cor.execute(list_tables)
    res = cor.fetchall()
    # if table exists don't do anything replace 'tablename' with your table name
    if 'tablename' in res:
        con.close()
        return None
    else:
        # create table schema here
        pass

In [418]:
def clean_csv_and_push_to_db(csv_save_path:PurePath, sqlite_file_path:PurePath):
    # clean data e.g make date times in the right format, 
    
    # fix names of columns into snake_case
    
    # push data to db
    con = sqlite3.connect(sqlite_file_path)

In [419]:
def create_metrics(sqlite_file_path:PurePath, processed_rows:int):
    # create metrics about the dataset to check for 
    # data freshness (when you wan to know the last time the data was updated)
    # logic test (is data realistic?)
    # number of rows processed vs unique rows
    # use logging info, warning, critical,etc... to relate if the metrics are good, bad, or really bad
    pass

In [420]:
# list of the html header and data 
def html_header_data(html_path:str):
    # empty list 
    data = []

    # for getting the header from 
    # the HTML file 
    list_header = [] 
    soup = BeautifulSoup(open(html_path),'html.parser') 
    header = soup.find_all(class_="title-transactionInfo") 
    for item in header: 
        try: 
            list_header.append(item.get_text().replace("-", '').strip().replace(' ', '_').lower()) 
        except: 
            continue
    # for getting the data  
    HTML_data = soup.find_all(class_="TransactionBoxInfo")

    for element in HTML_data: 
        sub_data = '' 
        for sub_element in element: 
            try: 
                if sub_element.get_text() == '':
                    sub_element = 'NA'
                else:
                   sub_element = sub_element.get_text().replace('\n', '').replace('\t', '').strip()
                sub_data += sub_element 
            except: 
                continue
        data.append(sub_data)

    # return the header and data
    return (list_header, data)

# ETL Orchestration

In [421]:
processed_rows = scrape_html_to_csv(html_folder_path)
print(processed_rows)
initalize_tables(sqlite_file_path)
clean_csv_and_push_to_db(csv_save_path, sqlite_file_path)
create_metrics(sqlite_file_path,processed_rows)

1000
